In [1]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [2]:
function pw(n, x::T) where {T}
    if iszero(n)
        one(T)
    elseif isodd(n)
        x * pw(n - 1, x)
    else
        r = pw(n ÷ 2, x)
        r * r
    end
end

pw (generic function with 1 method)

In [3]:
Tuple(pw(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [4]:
function pw_pe(::Val{n}, x::T) where {n,T}
    if iszero(n)
        one(T)
    elseif isodd(n)
        x * pw_pe(Val(n - 1), x)
    else
        r = pw_pe(Val(n ÷ 2), x)
        r * r
    end
end

pw_pe (generic function with 1 method)

In [5]:
Tuple(pw_pe(Val(n), "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [6]:
@code_typed pw_pe(Val(5), 10)

CodeInfo(
1 ─      nothing::Nothing
│   %2 = Base.mul_int(x, 1)::Int64
│   %3 = Base.mul_int(%2, %2)::Int64
│   %4 = Base.mul_int(%3, %3)::Int64
│   %5 = Base.mul_int(x, %4)::Int64
└──      return %5
) => Int64

Лишнее умножение на `1`!

Можно побороть с помощью абстрактной интерпретации. Но специализатор в Джулии не умеет работать с частично статическими значениями. Поэтому, придётся работать с парой: (абстрактное значение, конкретное значение).

In [7]:
function pw_abs(n, x::T) where {T}
    if iszero(n)
        true, one(T)
    elseif isodd(n)
        r_one, r = pw_abs(n - 1, x)
        false, r_one ? x : x * r
    else
        r_one, r = pw_abs(n ÷ 2, x)
        r_one, r_one ? one(T) : r * r
    end
end

pw_abs (generic function with 1 method)

In [8]:
Tuple(pw_abs(n, "Abc")[2] for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [9]:
function pw_abs_pe(::Val{n}, x::T) where {n, T}
    if iszero(n)
        Val(true), one(T)
    elseif isodd(n)
        r_one, r = pw_abs_pe(Val(n - 1), x)
        Val(false), r_one isa Val{true} ? x : x * r
    else
        r_one, r = pw_abs_pe(Val(n ÷ 2), x)
        r_one, r_one isa Val{true} ? one(T) : r * r
    end
end

pw_abs_pe (generic function with 1 method)

In [10]:
Tuple(pw_abs_pe(Val(n), "Abc")[2] for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [11]:
@code_typed pw_abs_pe(Val(5), 10)

CodeInfo(
1 ─      nothing::Nothing
│   %2 = Base.mul_int(x, x)::Int64
│   %3 = Base.mul_int(%2, %2)::Int64
│   %4 = Base.mul_int(x, %3)::Int64
│   %5 = Core.tuple($(QuoteNode(Val{false}())), %4)::Tuple{Val{false}, Int64}
└──      return %5
) => Tuple{Val{false}, Int64}

Теперь попробуем сделать то же самое с помощью `@generated`.

In [12]:
function pw_gen_impl1(n)
    if iszero(n)
        :(1.0)
    elseif isodd(n)
        :(x * $(pw_gen_impl1(n - 1)))
    else
        r = pw_gen_impl1(n ÷ 2)
        :($r * $r)
    end
end

pw_gen_impl1 (generic function with 1 method)

In [13]:
pw_gen_impl1(5) |> cleanup

:(x * (((x * 1.0) * (x * 1.0)) * ((x * 1.0) * (x * 1.0))))

Недостатки: (1) умножение на 1.0 и (2) дублирование кода.

In [14]:
@generated function pw_gen1(::Val{n}, x::Float64) where {n}
    pw_gen_impl1(n)
end

pw_gen1 (generic function with 1 method)

In [15]:
Tuple(pw_gen1(Val(n), 10.0) for n in 0:5)

(1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0)

In [16]:
function pw_gen_impl2(n)
    if iszero(n)
        :(1.0)
    elseif isodd(n)
        :(x * $(pw_gen_impl2(n - 1)))
    else
        quote
            r = $(pw_gen_impl2(n ÷ 2))
            r * r
        end
    end
end

pw_gen_impl2 (generic function with 1 method)

In [17]:
pw_gen_impl2(5) |> cleanup

:(x * begin
          r = begin
                  r = x * 1.0
                  r * r
              end
          r * r
      end)


Недостатки: (1) умножение на 1.0 и (2) слишком много вложенных `begin ... end`.

А хочется получить линейную программу. Но для этого придётся генерировать имена переменных!

Немного продвигаемся  в сторону линейной программы.

In [18]:
function pw_gen_impl3(n)
    if iszero(n)
        :(1.0)
    elseif isodd(n)
        quote
            r = $(pw_gen_impl3(n - 1))
            x * r
        end
    else
        quote
            r = $(pw_gen_impl3(n ÷ 2))
            r * r
        end
    end
end

pw_gen_impl3 (generic function with 1 method)

In [19]:
pw_gen_impl3(5) |> cleanup

quote
    r = begin
            r = begin
                    r = begin
                            r = 1.0
                            x * r
                        end
                    r * r
                end
            r * r
        end
    x * r
end

In [20]:
@generated function pw_gen3(::Val{n}, x::Float64) where {n}
    pw_gen_impl3(n)
end

pw_gen3 (generic function with 1 method)

In [21]:
Tuple(pw_gen3(Val(n), 10.0) for n in 0:5)

(1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0)

Делаем генерацию переменных и генерацию линейной программы.

In [22]:
function pw_gen_impl4!(n, es, m)
    r = Symbol("r_", m[])
    m[] += 1

    if iszero(n)
        push!(es, :($r = 1.0))
    elseif isodd(n)
        r1 = pw_gen_impl4!(n - 1, es, m)
        push!(es, :($r = x * $r1))
    else
        r1 = pw_gen_impl4!(n ÷ 2, es, m)
        push!(es, :($r = $r1 * $r1))
    end

    return r
end

pw_gen_impl4! (generic function with 1 method)

In [23]:
function pw_gen_impl4(n)
    es = Expr[]
    m = Ref(1)
    r = pw_gen_impl4!(n, es, m)

    quote
        $(es...)
        return $r
    end
end

pw_gen_impl4 (generic function with 1 method)

In [24]:
@generated function pw_gen4(::Val{n}, x::Float64) where {n}
    pw_gen_impl4(n)
end

pw_gen4 (generic function with 1 method)

In [25]:
Tuple(pw_gen4(Val(n), 10.0) for n in 0:5)

(1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0)

In [26]:
pw_gen_impl4(5) |> cleanup

quote
    r_5 = 1.0
    r_4 = x * r_5
    r_3 = r_4 * r_4
    r_2 = r_3 * r_3
    r_1 = x * r_2
    return r_1
end

Недостатки: (1) умножение на `1.0`, (2) лишние присваивания.

Побороть можно с помощью абстрактной интерпретации: (1) отслеживая, когда происходит умножение на `1.0` и (2) отслеживая, когда символическим значением переменной является число или идентификатор.

Но в данном случае можно обойтись без создания особого типа данных для абстрактных значений, поскольку в Джулии константные выражения представлены просто константами.

In [27]:
mult_s(x, y::Float64) =
    y == 1.0 ? x : :($x * $y)

mult_s(x, y) =
    :($x * $y)

mult_s (generic function with 2 methods)

In [28]:
function ass!(es, d, s::Symbol, c::Float64)
    d[s] = c
end

function ass!(es, d, s::Symbol, u::Symbol)
    d[s] = u
end

function ass!(es, d, s::Symbol, e)
    d[s] = s
    push!(es, :($s = $e))
end

ass! (generic function with 3 methods)

In [29]:
function pw_gen_impl5!(n, d, es, m)
    r = Symbol("r_", m[])
    m[] += 1

    if iszero(n)
        ass!(es, d, r, 1.0)
    elseif isodd(n)
        r1 = pw_gen_impl5!(n - 1, d, es, m)
        ass!(es, d, r, mult_s(:x, r1))
    else
        r1 = pw_gen_impl5!(n ÷ 2, d, es, m)
        ass!(es, d, r, mult_s(r1, r1))
    end

    return d[r]
end

pw_gen_impl5! (generic function with 1 method)

In [30]:
function pw_gen_impl5(n)
    d = Dict{Symbol,Any}()
    es = Expr[]
    m = Ref(1)
    r = pw_gen_impl5!(n, d, es, m)

    quote
        $(es...)
        return $r
    end
end

pw_gen_impl5 (generic function with 1 method)

In [31]:
@generated function pw_gen5(::Val{n}, x::Float64) where {n}
    pw_gen_impl5(n)
end

pw_gen5 (generic function with 1 method)

In [32]:
Tuple(pw_gen5(Val(n), 10.0) for n in 0:5)

(1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0)

In [33]:
pw_gen_impl5(5) |> cleanup

quote
    r_3 = x * x
    r_2 = r_3 * r_3
    r_1 = x * r_2
    return r_1
end

In [34]:
pw_gen_impl5(0) |> cleanup

:(return 1.0)

In [35]:
pw_gen_impl5(1) |> cleanup

:(return x)

In [36]:
pw_gen_impl5(2) |> cleanup

quote
    r_1 = x * x
    return r_1
end